# out_fiberID
* 기존변수 + u-g,g-r,r-i,i-z만 추가변수로 사용
* fiber_ID를 제외하고 진행해봄
* 제출결과 : 0.3793087274 
-> fiberID를 제외해서 0.02상승

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss


import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV, GridSearchCV

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# #colab용
# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/Dacon/천체관측/')

In [4]:
# path = "/content/drive/My Drive/Colab Notebooks/Dacon/천체관측/data/" #colab
path = "./data/" #local
train_df = pd.read_csv(path+'train.csv',index_col=0)
test_df = pd.read_csv(path+'test.csv',index_col=0)
sample_submission_df = pd.read_csv(path+'sample_submission.csv',index_col=0)
pd.options.display.max_columns = 30

In [5]:
train_df.shape, test_df.shape

((199991, 22), (10009, 21))

## DATA Setting

In [6]:
def del_outlier(data, min=0, max=60):
    up_idx_t=()
    dw_idx_t=()
    train_light = data.iloc[:,2:]
    for i in range(len(train_light.columns)):
        col = train_light.columns[i]
        up_idx_t+=tuple(data[data[col]>max].index)
        dw_idx_t+=tuple(data[data[col]<min].index)
    del_idx = set(up_idx_t+dw_idx_t)
    
    return data[~data.index.isin(del_idx)]

In [7]:
train_df = del_outlier(train_df, min=-60, max=60)

In [8]:
train_df.shape

(199805, 22)

In [9]:
column_number = {}
number_columns = {}
for i, column in enumerate(sample_submission_df.columns):
    column_number[column] = i
    number_columns[i] = column
    
    
def to_number(x, dic):
    return dic[x]

train_df['type_num'] = train_df['type'].apply(lambda x: to_number(x, column_number))


In [10]:
def add_minus_feature_all(data,test = False):
    '''
    combination으로 모든 변수들의 조합에 대해서 컬럼생성 
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 64개로 증가
    '''
    from itertools import combinations
    n = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for combi in list(combinations(selected,2)):
            name_1st = str.split(combi[0],'_')[1]
            name_2nd = str.split(combi[1],'_')[1]
            data[mag+"_"+name_1st+"-"+name_2nd] = data[combi[0]]-data[combi[1]]
        n=count
    return data

In [11]:
def add_minus_feature(data,test = False):
    '''
    근접변수에 대해서만 컬럼생성 ex) 'fiberMag_u-g', 'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z'
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 38개로 증가 
    '''
    from itertools import combinations
    n = 0
    m = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for i in range(0,len(selected)):        
            if i>0: 
                name_1st = str.split(m,'_')[1]
                name_2nd = str.split(selected[i],'_')[1]
                data[mag+"_"+name_1st+"-"+name_2nd] = data[m]-data[selected[i]]
            m = selected[i]    
        n=count
    return data

In [12]:
train_df = add_minus_feature_all(train_df)
test_df = add_minus_feature_all(test_df,True)

In [13]:
train_df.shape,test_df.shape

((199805, 63), (10009, 61))

In [14]:
train_df.columns

Index(['type', 'fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
       'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
       'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
       'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
       'modelMag_z', 'type_num', 'psfMag_u-g', 'psfMag_u-r', 'psfMag_u-i',
       'psfMag_u-z', 'psfMag_g-r', 'psfMag_g-i', 'psfMag_g-z', 'psfMag_r-i',
       'psfMag_r-z', 'psfMag_i-z', 'fiberMag_u-g', 'fiberMag_u-r',
       'fiberMag_u-i', 'fiberMag_u-z', 'fiberMag_g-r', 'fiberMag_g-i',
       'fiberMag_g-z', 'fiberMag_r-i', 'fiberMag_r-z', 'fiberMag_i-z',
       'petroMag_u-g', 'petroMag_u-r', 'petroMag_u-i', 'petroMag_u-z',
       'petroMag_g-r', 'petroMag_g-i', 'petroMag_g-z', 'petroMag_r-i',
       'petroMag_r-z', 'petroMag_i-z', 'modelMag_u-g', 'modelMag_u-r',
       'modelMag_u-i', 'modelMag_u-z', 'modelMag_g-r', 'modelMag_g-i',
       'modelMag_g-z', 'modelMag_r-i', 'modelMag_r-z

### CV

In [54]:
X = train_df.drop(columns=['type', 'type_num'], axis=1)
y = train_df['type_num']
test_X = test_df

In [53]:
import sklearn
logloss = sklearn.metrics.make_scorer(log_loss,needs_proba=True, labels=y)

In [55]:
folds = 3
Fold = StratifiedKFold(n_splits=folds, shuffle=True,random_state=42)
kfold =KFold(n_splits=folds, shuffle=True,random_state=42)

In [24]:
params = {"max_depth" : (7,8,9),
         "learning_rate": (0.01,0.05, 0.1),
         "num_leaves": (127,255,511),
         "min_child_samples": (30,40,50),
         "lambda_l1": (0,3,5,7,9),
         "lambda_l2": (0,3,5,7,9)}

In [56]:
defaut_lgbmc = lgb.LGBMClassifier(n_estimators=1500,random_state=1029,objective ="multiclass",class_weight = 'balanced')

In [57]:
random_search  = RandomizedSearchCV(defaut_lgbmc,param_distributions=params, cv=5, verbose =100 ,n_iter = 10,scoring=logloss)

In [58]:
#와 3시간.. fold = 3,cv =5, n_iter =2 
for fold, (train_ids, val_ids) in enumerate(Fold.split(X,y)):
    print("★", fold, "start")
    eval_set = [(X.iloc[train_ids], y.iloc[train_ids]),(X.iloc[val_ids], y.iloc[val_ids])]
    eval_names = ['train','val']
    random_search.fit(X, y, eval_set = eval_set, eval_names = eval_names, early_stopping_rounds = 100,verbose = 100)

★ 0 start
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] num_leaves=127, min_child_samples=40, max_depth=7, learning_rate=0.05, lambda_l2=7, lambda_l1=5 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.48403	val's multi_logloss: 0.489345
Did not meet early stopping. Best iteration is:
[100]	train's multi_logloss: 0.48403	val's multi_logloss: 0.489345
[CV]  num_leaves=127, min_child_samples=40, max_depth=7, learning_rate=0.05, lambda_l2=7, lambda_l1=5, score=0.510, total= 1.9min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s
[CV] num_leaves=127, min_child_samples=40, max_depth=7, learning_rate=0.05, lambda_l2=7, lambda_l1=5 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.485849	val's multi_logloss: 0.489638
Did not meet early stopping. Best iteration is:
[100]	train's

KeyboardInterrupt: 

In [28]:
best_params = random_search.best_params_
best_score = random_search.best_score_
best_model = random_search.best_estimator_

In [ ]:
with open('best_pram.txt', 'w' ) as best:
    best.write("best_params : " + str(best_params) + "\n")
    best.write("best_score : " + str(best_score)+"\n")
    best.write("best_model : " + str(best_model)+"\n")

In [32]:
print(random_search.best_score_)

0.8561597556079719

In [30]:
gmodel = random_search.best_estimator_

In [31]:
print(gmodel)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', lambda_l1=9,
               lambda_l2=3, learning_rate=0.1, max_depth=8,
               min_child_samples=40, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=255,
               objective='multiclass', random_state=1029, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [33]:
gmodel.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', lambda_l1=9,
               lambda_l2=3, learning_rate=0.1, max_depth=8,
               min_child_samples=40, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=255,
               objective='multiclass', random_state=1029, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [38]:
train_y_pred = gmodel.predict_proba(X)

In [40]:
train_y_pred

array([[4.56201164e-06, 7.75154813e-06, 2.41460155e-06, ...,
        2.06569761e-06, 5.54613973e-06, 3.44935629e-04],
       [1.50541897e-05, 4.09494448e-05, 4.57933724e-06, ...,
        5.43068514e-06, 1.01243090e-05, 1.80857595e-02],
       [6.38381018e-04, 9.48061368e-05, 2.43798597e-05, ...,
        4.43212257e-05, 5.53072624e-05, 5.09389605e-02],
       ...,
       [1.97054983e-05, 1.38945056e-05, 8.67211163e-05, ...,
        2.49992402e-05, 1.10094056e-05, 1.15205513e-05],
       [3.61167959e-06, 2.22046100e-06, 6.81626731e-07, ...,
        8.98888354e-07, 1.16654560e-05, 5.28685738e-06],
       [8.87847708e-04, 8.46730369e-04, 5.83058098e-05, ...,
        6.31506477e-05, 3.47337857e-04, 5.39072266e-01]])

In [34]:
y_pred = gmodel.predict_proba(test_X)

In [35]:
y_pred

array([[3.50066638e-05, 5.26650580e-05, 2.21850300e-04, ...,
        1.92954943e-04, 1.90364475e-04, 3.46413745e-05],
       [8.82200088e-05, 1.90074363e-04, 2.24943431e-05, ...,
        9.93321181e-05, 1.38933440e-04, 2.00775835e-04],
       [2.71777282e-05, 1.13690184e-05, 2.20370274e-06, ...,
        5.10410746e-06, 1.50041203e-05, 1.08798329e-05],
       ...,
       [9.31616009e-05, 2.19138193e-04, 1.27084447e-05, ...,
        1.72416868e-05, 2.77744878e-05, 9.95822677e-02],
       [1.05948715e-05, 1.29974202e-05, 1.80738664e-06, ...,
        1.57848834e-06, 3.77686680e-06, 3.16995458e-04],
       [1.53532987e-04, 3.88460633e-06, 1.68581882e-06, ...,
        2.56362557e-06, 3.18271400e-06, 9.00399895e-06]])

In [39]:
log_loss = log_loss(y,train_y_pred,labels= list(set(y)))

0.2059535600699039

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
submission.to_csv('submission.csv', index=True)

## Modeling